# Stupid Tensorflow Tricks, II
Solving an old physics problem with Tensorflow!
_part one [here](https://towardsdatascience.com/stupid-tensorflow-tricks-3a837194b7a0)_ 

## Brachistocrone problem
What's the fastest way down a ramp? We are going to solve for the height of the green dots that minimize the time.

![](figures/plane.png)

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

# This lets us see computations as soon as we run them!
tf.enable_eager_execution()

1.14.0


In [2]:
# Setup the "ramp"
N = 20
width = 3
y = tf.Variable(np.linspace(0.99, 0.01, N,), dtype=tf.float32)
x = tf.linspace(0.0, 1.0, N+2) * width
print(y)

<tf.Variable 'Variable:0' shape=(20,) dtype=float32, numpy=
array([0.99      , 0.9384211 , 0.88684213, 0.83526313, 0.7836842 ,
       0.73210526, 0.6805263 , 0.6289474 , 0.57736844, 0.5257895 ,
       0.47421053, 0.4226316 , 0.37105262, 0.31947368, 0.26789474,
       0.21631579, 0.16473684, 0.1131579 , 0.06157895, 0.01      ],
      dtype=float32)>


### Conservation of Energy
$$U=K$$
$$mgh = \frac{1}{2}mv^2$$
$$v = \sqrt{2gh}$$

In [3]:
g = tf.constant(9.8)
v = tf.sqrt(2 * g * (1 - y));
print(v)

tf.Tensor(
[0.44271865 1.0986114  1.4892597  1.7968982  2.059075   2.2914488
 2.5023358  2.6967816  2.8781204  3.0486925  3.2102141  3.3639889
 3.5110354  3.652166   3.7880425  3.9192102  4.0461287  4.169185
 4.2887125  4.4049973 ], shape=(20,), dtype=float32)


# Time to roll down 
can compute for each little segment...
$$ z(t) = z_0 + v_0 t + \frac{1}{2}a_z t^2 = z_0 + v_0 t + \frac{g \sin(\theta) t^2}{2} $$


$$ t = \frac{-v_0 \pm \sqrt{v_0^2 - 2 g \sin(\theta)z_0}} {g \sin(\theta)} $$

Ugh. We have to keep track of the root: the first root if going up the ramp, the second root if we are moving up the ramp.

... that's stupid and too much work. 

## Lazy physics ...
Over a short enough time span we can ignore gravity, as long as we fix the velocity each segment.

$$ z(t) = z_0 + v_0 t $$
$$ t = z_0 / v_0  $$

The change along the ramp:

![](figures/plane_crop.png)

$$ t = \sqrt{\frac{\Delta x^2}{\Delta y^2}} \frac{1}{v_0}  $$

In [4]:
dx = x[1] - x[0]
dy = y[:-1] - y[1:]
dz = tf.sqrt(dx ** 2 + dy ** 2)
print(dz)

tf.Tensor(
[0.15188335 0.15188335 0.15188336 0.15188335 0.15188335 0.15188335
 0.15188335 0.15188335 0.15188335 0.15188335 0.15188335 0.15188335
 0.15188335 0.15188335 0.15188335 0.15188335 0.15188335 0.15188335
 0.15188335], shape=(19,), dtype=float32)


Let's clean it up, and encapsulate it all in a function.

In [5]:
def compute_ramp(y):
    g = tf.constant(9.8)
    
    # Fix the endpoints
    y = tf.concat([[1.0], y, [0.0]], axis=0)
    
    dx = x[1] - x[0]
    dy = y[:-1] - y[1:]

    v = tf.sqrt(2 * g * (y[0] - y))
    avg_v = (v[1:] + v[:-1]) / 2

    dz = tf.sqrt(dx ** 2 + dy ** 2)

    total_time = tf.reduce_sum(dz / avg_v)
    return total_time


In [6]:
# For a 3x1 ramp, this should be ~ 1.34s
print(compute_ramp(y).numpy())

1.8699452


In [7]:
# Oof. Let's try it with more points!
N = 2000
width = 3
y = tf.Variable(np.linspace(0.99, 0,N,), dtype=tf.float32)
x = tf.linspace(0.0, 1.0, N+2) * width
print(compute_ramp(y).numpy())

1.3422576
